In [1]:
import os
import re
import tqdm
import string
import pandas as pd
import numpy as np
import util
from sklearn.decomposition import TruncatedSVD
#from data_preprocessing import *
import word_embedding_load as wel
import baseline_classification as bc
import matplotlib.pyplot as plt
import feature_selection_evaluation as fse
import pickle
from xgboost_classifier import *
from testaccuracy import *

C:\Users\zhuya\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\zhuya\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
modelName = 'docEmbeddings_win2_load_all.d2v'

In [ ]:
res = fse.getFeature(modelName, 100)
X_100_train = res['X_train']
X_100_test = res['X_test']
y = res['label']

In [6]:
res = fse.getFeature(modelName, 175)
X_175_train = res['X_train']
X_175_test = res['X_test']

Successfully loaded the textmodel from ../model/doc2vec/docEmbeddings_win2_load_all.d2v
Feature with length 175 using docEmbeddings_win2_load_all.d2v


In [7]:
print(X_100_train.shape)
print(X_100_test.shape)
print(X_175_train.shape)
print(X_175_test.shape)
print(y.shape)

(3689, 100)
(986, 100)
(3689, 175)
(986, 175)
(3689,)


In [6]:
if not os.path.isdir('../data/feature'):
    os.mkdir('../data/feature')


In [8]:
 #save feature data to ../data/feature/

with open('../data/feature/docFeat_100_train.pkl', 'wb') as f:
    pickle.dump(X_100_train, f)
    f.close()
with open('../data/feature/docFeat_175_train.pkl', 'wb') as f:
    pickle.dump(X_175_train, f)
    f.close()
with open('../data/feature/docFeat_100_test.pkl', 'wb') as f:
    pickle.dump(X_100_test, f)
    f.close()
with open('../data/feature/docFeat_175_test.pkl', 'wb') as f:
    pickle.dump(X_175_test, f)
    f.close()
with open('../data/feature/docLabel.pkl', 'wb') as f:
    pickle.dump(y, f)
    f.close()


In [2]:
# load feature data from earlier computed 
with open('../data/feature/docFeat_100_train.pkl', 'rb') as f:
    X_100_train = pickle.load(f)
    f.close()
with open('../data/feature/docFeat_175_train.pkl', 'rb') as f:
    X_175_train = pickle.load(f)
    f.close()
with open('../data/feature/docFeat_100_test.pkl', 'rb') as f:
    X_100_test = pickle.load(f)
    f.close()
with open('../data/feature/docFeat_175_test.pkl', 'rb') as f:
    X_175_test = pickle.load(f)
    f.close()
with open('../data/feature/docLabel.pkl', 'rb') as f:
    y = pickle.load(f)
    f.close()

In [3]:
print(X_100_train.shape)
print(X_100_test.shape)
print(X_175_train.shape)
print(X_175_test.shape)
print(y.shape)

(3689, 100)
(986, 100)
(3689, 175)
(986, 175)
(3689,)


In [4]:
[all_data, train_size, test_size, train_x, train_y, test_x] = util.loadData()
for i in range(len(train_y)):
    train_y[i] -=1  

In [5]:
y_predict = xgbclassifier(X_175_train, train_y, X_175_test, 1, 8, 1000)

[0]	train-mlogloss:2.12065	valid-mlogloss:2.14319
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[25]	train-mlogloss:1.08116	valid-mlogloss:1.47622
[50]	train-mlogloss:0.641457	valid-mlogloss:1.22074
[75]	train-mlogloss:0.409081	valid-mlogloss:1.10375
[100]	train-mlogloss:0.275661	valid-mlogloss:1.04461
[125]	train-mlogloss:0.195875	valid-mlogloss:1.01644
[150]	train-mlogloss:0.141437	valid-mlogloss:1.00516
[175]	train-mlogloss:0.10504	valid-mlogloss:1.00261
[200]	train-mlogloss:0.07972	valid-mlogloss:1.00707
[225]	train-mlogloss:0.061824	valid-mlogloss:1.0127
[250]	train-mlogloss:0.049253	valid-mlogloss:1.0204
Stopping. Best iteration:
[174]	train-mlogloss:0.106197	valid-mlogloss:1.00255

final validation loss:
1.0025462156
final validation accuracy:
0.66015037594


In [6]:
savesubmisstion(y_predict, test_x, filename = "submission_svd175.csv")